In [1]:
!pip install ipython-autotime

In [2]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
train_data = pd.read_csv("twitter_training.csv")
train_data = train_data.dropna()

In [4]:
for col in train_data.columns:
    print(col)

2401
Borderlands
Positive
im getting on borderlands and i will murder you all ,


In [5]:
train_data.rename(columns={2401:'id','Borderlands':'name','Positive':'sentiment','im getting on borderlands and i will murder you all ,':'text'},inplace=True)

In [6]:
mapping = {
    'Positive': 1,
    'Negative': 0,
    'Neutral': 2,
    'Irrelevant': 4
}

In [7]:
train_data['sentiment_label'] = [mapping[value] for value in train_data['sentiment']]

In [8]:
train_data.drop(['2401','name'],axis=1)

,sentiment,text,sentiment_label
0,Positive,I am coming to the borders and I will kill you...,1
1,Positive,im getting on borderlands and i will kill you ...,1
2,Positive,im coming on borderlands and i will murder you...,1
3,Positive,im getting on borderlands 2 and i will murder ...,1
4,Positive,im getting into borderlands and i can murder y...,1
...,...,...,...
74676,Positive,Just realized that the Windows partition of my...,1
74677,Positive,Just realized that my Mac window partition is ...,1
74678,Positive,Just realized the windows partition of my Mac ...,1
74679,Positive,Just realized between the windows partition of...,1


In [9]:
import re 
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

def preprocess_text(text):
    if not isinstance(text, str):
        return "" # Return an empty string for non-string values
    text=re.sub('<[^>]*>','',text)
    text=re.sub('[^a-zA-Z]',' ',text).lower()
    words = word_tokenize(text)
    stop_words= set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    preprocessed_text = ' '.join(words)
    return preprocessed_text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\duong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\duong\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\duong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\duong\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
train_data['preprocessed_text']=train_data['text'].apply(preprocess_text)

In [11]:
train_data.nunique()

2401                 12447
name                    32
sentiment                4
text                 69490
sentiment_label          4
preprocessed_text    59828
dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_data['preprocessed_text'],train_data['sentiment_label'],test_size=0.2,random_state=45)

In [13]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
vectorizer1=CountVectorizer()
vectorizer2=TfidfVectorizer()
X_train_count=vectorizer1.fit_transform(X_train)
X_test_count=vectorizer1.transform(X_test)
X_train_tfidf=vectorizer2.fit_transform(X_train)
X_test_tfidf=vectorizer2.transform(X_test)
clf1=SVC()
clf1.fit(X_train_count,y_train)
y_predict_count=clf1.predict(X_test_count)
accuracy_count=accuracy_score(y_test,y_predict_count)
print("Prediction accuracy in COUNT is : ",round(accuracy_count,3)*100,'%')
clf2 = SVC()
clf2.fit(X_train_tfidf, y_train)
y_predict_tfidf = clf2.predict(X_test_tfidf)
accuracy_tfidf=accuracy_score(y_test,y_predict_tfidf)
print("Prediction accuracy in tfidf is : ",round(accuracy_tfidf,3)*100,'%')

Prediction accuracy in COUNT is :  86.5 %
Prediction accuracy in tfidf is :  92.2 %


In [14]:
t="During Mohammed Azharuddin's time, even Zimbabwe seemed tough for India. Then MK Dhoni came and we started defeating Australia. And we because the world champions. When Manmohan Singh was the PM, even Pakistan seemed powerful. Today, after Modi's arrival we find China to be a lightweight. What's important is who is your captain."
t=preprocess_text(t)
t=[t]

In [15]:
t2=vectorizer2.transform(t)

In [16]:
pred1 = clf2.predict(t2)
if pred1 == 0:
    print('Negative')
elif pred1 == 1:
    print('Positive')
elif pred1 == 2:
    print('Neutral')
else:
    print('Irrelevant')

Neutral
